In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
feb_template_path = Path("../sample_files/sample_lv_template_mod.feb")

# Simple creation from .feb:

For a quick and easy LV creation from .feb, you can use the "from_file" or "from_feb" methods. They load a feb object and parses the information into our library's data structure. However, remember that .feb file can be complex, so this method only loads the first set of nodes and elements, nodesets and surfaces. Everything else is ignored (as of current version).

In [3]:
lv = LV.from_file(feb_template_path)

Let's create a LV region from .feb nodesets and plot:

In [4]:
lv.set_region_from_nodesets(LV_MESH_DATA.EPI_ENDO, ["ENDO", "EPI"])
lv.plot(scalars=LV_MESH_DATA.EPI_ENDO, categorical=True)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

Here is another example:

In [5]:
lv.set_region_from_nodesets("EEBB", ["ENDO", "EPI", "BORDER_MITRAL", "BORDER_AORTIC"])
lv.plot(scalars="EEBB", categorical=True)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

# Fine control object creation from .feb

If you would like a finer control of what data is being parsed from .feb and added to the LV object, you can manually load an feb object and use our build-function to add data from .feb to our LV object. Here is an example:

Load feb object using febio_python library.

In [6]:
from febio_python.feb import FEBio_feb
feb = FEBio_feb.from_file(feb_template_path)
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 24
--> Nodes 'LV': 64452
--> Nodes 'AORTIC_RIM': 128
--> Nodes 'MITRAL_RIM': 128
--> Elements 'LV': 218359
--> Elements 'AORTIC_RIM_ELEMS': 63
--> Elements 'MITRAL_RIM_ELEMS': 63
-> Boundary: 1
-> Loads: 1
-> Discrete: 3
-> LoadData: 3
-> Output: 1
-> MeshData: 1

Get nodes and elements dictionaries. 

In [7]:
nodes = feb.get_nodes()
print(nodes.keys())
elements = feb.get_elements()
print(elements.keys())

dict_keys(['LV', 'AORTIC_RIM', 'MITRAL_RIM'])
dict_keys(['LV', 'AORTIC_RIM_ELEMS', 'MITRAL_RIM_ELEMS'])


Select what set of nodes and elements to be used to build our LV object. Remember that in .feb files, elements start at 1, so we will have to correct it:

In [8]:
lvnodes = nodes["LV"]
lvelems = elements["LV"] - 1 # feb elements starts at 1

Get nodesets and surfaces. Other data can also be extract from feb object. It is structures as a xml tree; check febio_python for more details.

In [9]:
nodesets = feb.get_nodesets()
print(nodesets.keys())
surfaces = feb.get_surfaces()
print(surfaces.keys())

dict_keys(['ENDO', 'EPI', 'AORTIC', 'ENDO_AORTIC', 'EPI_AORTIC', 'BORDER_AORTIC', 'MITRAL', 'ENDO_MITRAL', 'EPI_MITRAL', 'BORDER_MITRAL', 'AM_INTERCECTION', 'ENDO_AM_INTERCECTION', 'EPI_AM_INTERCECTION', 'BASE_REGION', 'APEX_REGION'])
dict_keys(['ENDO'])


Now let's build our LV object:

In [10]:
lv = LV.from_nodes_elements(nodes=lvnodes, elements=lvelems)

add nodesets and surface sets:

In [11]:
for key, value in nodesets.items():
  lv.add_nodeset(key, value)

for key, value in surfaces.items():
  lv.add_surface_oi(key, value)

Let's create a LV region for plotting:

In [12]:
lv.set_region_from_nodesets(LV_MESH_DATA.EPI_ENDO, ["ENDO", "EPI"])
lv.plot(scalars=LV_MESH_DATA.EPI_ENDO)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…